# Seaborn y Pysolr

El objetivo de esta notebook se mostrar un ejemplo de análisis de datos que es posible hacer con el dataset luego del preprocesamiento, ingesta e indexación del mismo. 

Adicionalmente, se espera que el análisis tenga un buen tiempo de ejecución gracias a la indexación de la data hecha con Solr.

Se utilizá:
    - Seaborn, una potente librería de plotting.
    - Pysolr, una librería fácil de usar para hacer consultas a solr desde python.

In [ ]:
# Install dependencies
! conda install -c conda-forge pysolr

In [ ]:
import pysolr
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
sns.set(style="white", context="talk")
rcParams['figure.figsize'] = 11.7,11.7

hostname = "http://192.168.56.102"
port = "8983"
core = "movies_indexed" #Create that core before running this notebook

# Connect to solr
solrcon = pysolr.Solr(f'{hostname}:{port}/solr/{core}', timeout=100)

In [ ]:
# This method is needed for expanded tables creation. 
# Given a list of columns that contains list values, it creates a new row for each value of the list.
# Data has list attributes that seaborn requires to be in different rows for plotting.
# Also, solr returns every value inside of a list, which is unconvinient for processing.
def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res

# Method was extracted from: 
#    https://stackoverflow.com/questions/12680754/split-explode-pandas-dataframe-string-entry-to-separate-rows

# Ganancias y ratings por género
En esta sección se graficaran el promedio de ganancias y el promedio de rating de cada género, ordenado de menor a mayor. El dataset indexado contienen películas sin género o ganacias, las cuales se filtran con una filter query de solr.

In [ ]:
# Get the movies with a pysolr query. Then create a Pandas dataframe with the result.
results = solrcon.search('*:*', fq=['revenue:[1 TO *]', 'genres:["" TO *]'], rows=50000)
movies_with_revenue_query = pd.DataFrame(results.docs)
movies_with_revenue_query

In [ ]:
# Seaborn style configuration
sns.set(style="white", context="talk")
rcParams['figure.figsize'] = 11.7,11.7

# Explode attributes and select the interesting ones (could also be done with pysolr for better performance).
movies_with_revenue = explode(movies_with_revenue_query, ['revenue', 'title'])[['title','genres','revenue', 'rating']]
movies_with_revenue = explode(movies_with_revenue, ['genres'])
movies_with_revenue['rating'] = movies_with_revenue['rating'].apply(lambda x : x[0] if type(x) is list else x)

# Group and calculate
genres_rating_mean=movies_with_revenue.groupby(['genres'],as_index=False).mean().sort_values(by=['rating'])
genres_revenue_mean=movies_with_revenue.groupby(['genres'],as_index=False).mean().sort_values(by=['revenue'])

# Charts configuration
chart_revenue=sns.barplot(x=genres_revenue_mean['genres'],y=genres_revenue_mean['revenue'])
chart_revenue.set_xticklabels(labels=genres_revenue_mean['genres'], rotation=90)
plt.show()

chart_revenue=sns.barplot(x=genres_rating_mean['genres'],y=genres_rating_mean['rating'])
chart_revenue.set_xticklabels(labels=genres_rating_mean['genres'], rotation=90)
plt.show()

# Ganancias y ratings por productora
En esta sección se graficaran el promedio de ganancias y el promedio de rating de cada productora, ordenado de menor a mayor. Dado que existen muchas productoras, solo se mostraran las 20 productoras con más peliculas ingresadas en el dataset (para que los promedios sean razonables).

In [ ]:
# Same process than before, get the data and explode columns of interest.
results = solrcon.search('*:*', fq=['revenue:[1 TO *]', 'production_companies:["" TO *]'], rows=50000)
query_df = pd.DataFrame(results.docs)

movies_with_revenue = explode(query_df, ['revenue', 'title'])[['title','production_companies', 'rating', 'revenue']]
movies_with_revenue = explode(movies_with_revenue, ['production_companies'])
movies_with_revenue['rating'] = movies_with_revenue['rating'].apply(lambda x : x[0] if type(x) is list else x)

companies_raitng_mean=movies_with_revenue.groupby(['production_companies'],as_index=False).mean().sort_values(by=['rating'], ascending=False)
companies_revenue_mean=movies_with_revenue.groupby(['production_companies'],as_index=False).mean().sort_values(by=['revenue'], ascending=False)
companies_movies_counts=movies_with_revenue.groupby(['production_companies'],as_index=False).count().sort_values(by=['title'], ascending=False).head(20)

# Merge companies with most movies in the dataset with the dataset containing the mean values.
companies_with_several_movies_ratings = pd.merge(companies_raitng_mean,companies_movies_counts['production_companies'],on='production_companies',how='inner').sort_values(by=['rating'])
companies_with_several_movies_revenue = pd.merge(companies_revenue_mean,companies_movies_counts['production_companies'],on='production_companies',how='inner').sort_values(by=['revenue'], ascending=False)

In [ ]:
chart_companies_rating=sns.barplot(x=companies_with_several_movies_ratings['production_companies'],y=companies_with_several_movies_ratings['rating'])
chart_companies_rating.set_xticklabels(labels=companies_with_several_movies_ratings['production_companies'], rotation=90)
chart_companies_rating.set_title("Raitings of companies with most movies produced")
chart_companies_rating.set_xlabel("Companies")
plt.show()

In [ ]:
chart_companies_revenue=sns.barplot(y=companies_with_several_movies_revenue['production_companies'],x=companies_with_several_movies_revenue['revenue'])
chart_companies_revenue.set_yticklabels(labels=companies_with_several_movies_revenue['production_companies'])
chart_companies_revenue.set_title("Revenue of companies with most movies produced")
chart_companies_revenue.set_ylabel("Companies")
plt.show()

# Heatmaps
En esta sección se exploran algunos heatmaps. Los heatmaps son herramientas muy útiles para identificar insights en la data ya que permite cruzar dos variables y analizar cada parte por separado. Retomando lo visto en las secciones anteriores, se cruzará esta información para mostrar el promedio de ganancias y el promedio de ratings de cada productora en cada género.

In [ ]:
results = solrcon.search('*:*', fq=['revenue:[1 TO *]', 'production_companies:["" TO *]', 'genres:["" TO *]'], rows=50000)
query_df = pd.DataFrame(results.docs)

movies_with_revenue = explode(query_df, ['revenue', 'title'])[['title','production_companies', 'rating', 'revenue', 'genres']]
movies_with_revenue = explode(movies_with_revenue, ['production_companies'])
movies_with_revenue = explode(movies_with_revenue, ['genres'])
movies_with_revenue['rating'] = movies_with_revenue['rating'].apply(lambda x : x[0] if type(x) is list else x)

companies_and_genres_with_revenue = movies_with_revenue[['production_companies', 'genres', 'revenue']].groupby(['production_companies', 'genres'],as_index=False).mean().sort_values(by=['revenue'], ascending=False)
companies_and_genres_with_revenue = pd.merge(companies_and_genres_with_revenue,companies_with_several_movies_revenue['production_companies'].head(20),on='production_companies',how='inner').sort_values(by=['revenue'])

companies_and_genres_with_rating = movies_with_revenue[['production_companies', 'genres', 'rating']].groupby(['production_companies', 'genres'],as_index=False).mean().sort_values(by=['rating'], ascending=False)
companies_and_genres_with_rating = pd.merge(companies_and_genres_with_rating,companies_with_several_movies_ratings['production_companies'].head(20),on='production_companies',how='inner').sort_values(by=['rating'])


### Promedio de ganancias de cada productora separado por género

In [ ]:
companies_and_genres_with_revenue_pivots = companies_and_genres_with_revenue.pivot("production_companies", "genres", "revenue")
ax = sns.heatmap(companies_and_genres_with_revenue_pivots, xticklabels=True, yticklabels=True, cmap="Greens")
ax.set_title("Companies revenue per genre")
plt.show()

### Promedio de rating de cada productora separado por género

In [ ]:
companies_and_genres_with_rating_pivots = companies_and_genres_with_rating.pivot("production_companies", "genres", "rating")
ax = sns.heatmap(companies_and_genres_with_rating_pivots, xticklabels=True, yticklabels=True, cmap="PiYG")
ax.set_title("Companies rating per genre")
plt.show()

# Filtrando por país
Gracias a Solr, podemos rápidamente realizar este mismo análisis simplemente cambiando la query de búsqueda. Por ejemplo, solo tomando en cuenta películas con participación de alguna productora de UK.

In [ ]:
results = solrcon.search('production_countries:"United Kingdom"', fq=['revenue:[1 TO *]', 'genres:["" TO *]', 'production_companies:["" TO *]'], rows=50000)
query_df = pd.DataFrame(results.docs)

movies_with_revenue = explode(query_df, ['revenue', 'title'])[['title','production_companies', 'rating', 'revenue', 'genres']]
movies_with_revenue = explode(movies_with_revenue, ['production_companies'])
movies_with_revenue = explode(movies_with_revenue, ['genres'])
movies_with_revenue['rating'] = movies_with_revenue['rating'].apply(lambda x : x[0] if type(x) is list else x)

companies_revenue_mean=movies_with_revenue.groupby(['production_companies'],as_index=False).mean().sort_values(by=['revenue'], ascending=False)
companies_movies_counts=movies_with_revenue.groupby(['production_companies'],as_index=False).count().sort_values(by=['title'], ascending=False).head(20)
companies_with_several_movies_revenue = pd.merge(companies_revenue_mean,companies_movies_counts['production_companies'],on='production_companies',how='inner').sort_values(by=['revenue'], ascending=False)

companies_and_genres_with_revenue = movies_with_revenue[['production_companies', 'genres', 'revenue']].groupby(['production_companies', 'genres'],as_index=False).mean().sort_values(by=['revenue'], ascending=False)
companies_and_genres_with_revenue = pd.merge(companies_and_genres_with_revenue,companies_with_several_movies_revenue['production_companies'].head(20),on='production_companies',how='inner').sort_values(by=['revenue'])


In [ ]:
companies_and_genres_with_revenue_pivots = companies_and_genres_with_revenue.pivot("production_companies", "genres", "revenue")
ax = sns.heatmap(companies_and_genres_with_revenue_pivots, xticklabels=True, yticklabels=True, cmap="Greens")
ax.set_title("Companies revenue per genre")
plt.show()